In [ ]:
#подключаем нужные нам библиотеки, 
import pandas as pd
import matplotlib.pyplot as plt
import pdfkit
from io import BytesIO
import binascii
import smtplib
from email import encoders
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
#получаем необходимые данные
data = pd.read_csv("https://video.ittensive.com/python-advanced/data-9722-2019-10-14.utf.csv", delimiter=";")
#выделяем из данных результаты только 2018-2019 годов
data = data[data["YEAR"] == "2018-2019"]
#ищем лучшею школу по результатам ЕГЭ, выполняем сортировку по PASSES_OVER_220
#и берём первое значений
#сортировка занчений также позволяет вынести два самых малочисленных значения
#чтобы подписи поместились на графике
data_best = data.sort_values("PASSES_OVER_220", ascending=False).head(1)
#выполняем группировку по административному округу, предварительно уберём из этой серии данных
#  все слова, кроме первого, для того чтобы подписи данных были короче
data["AdmArea"] = data["AdmArea"].apply(lambda x: x.split(" ")[0])
#считаем общее число отличников, оно нужно для отчёта и для вывода доли
data_adm = data.groupby("AdmArea").sum()["PASSES_OVER_220"].sort_values()
total = data_adm.sum()
#создаём холст
fig = plt.figure(figsize=(11,6))
area = fig.add_subplot(1,1,1)
#на холст задаём список explode, список секторов
explode = [0]*len(data_adm)
#указываем меру их выноса из основной диаграммы
explode[0] = 0.4
explode[1] = 0.4

#создаём круговую диаграмму по округа
data_adm.plot.pie(ax = area,
                #в подписи передадим пустой набор данных по числу округов
                labels=[""]*len(data_adm),
                # выведем назавния
                label="Отличники по ЕГЭ",
                cmap="tab20",
                #создаём авто подписи данных, которые сформируем
                # из доли точного значения отличников по округам
                autopct=lambda x:int(round(total * x/100)),
                pctdistance=0.9,
                #список для выноса секторов
                explode=explode)
#справа выведем легенуд с подписями округов                
plt.legend(data_adm.index, bbox_to_anchor=(1.5,1,0.1,0))
#сохраняем диаграмму, как изображения для выставки изображения в отчёт
img = BytesIO()
plt.savefig(img)
# для выставки изображения в отчёт, преобразуем его в base64 - кодировку
#стандартным образом
img = 'data:image/png;base64,' + binascii.b2a_base64(img.getvalue(),
                            newline=False).decode("UTF-8")
#задаём настройку pandas по длине значения  в колонке
pd.set_option("display.max_colwidth", 1000)
#формируем html - отчёт со всеми данными
#задаём кодировку, заголовок
#вставим суммарное значение числа отличников, распределение по округам, 
#нашу диаграмму, и название лучшей школы
html = '''<html>
<head>
    <title>Результаты ЕГЭ Москвы: отличники</title>
    <meta charset="utf-8"/>
</head>
<body>
    <h1>Результаты ЕГЭ Москвы: отличники в 2018-2019 году</h1>
    <p>Всего: ''' + str(total) + '''</p>
    <img src="E:/ittensive/assets/''' + img + '''" alt="Отличники по округам"/>
    <p>Лучшая школа: ''' + str(data_best["EDU_NAME"].values[0]) + '''</p>
</body>
</html>'''
#из html - документа формируем pdf - отчёт, через pdfkit
#конфигурируем бинарный файл, задаём настройки
#размер страницы и вывод номера страницы, на каждой страницы
config = pdfkit.configuration(wkhtmltopdf="E:/programs/wk_html_pdf/wkhtmltopdf/bin/wkhtmltopdf.exe")
options = {
    'page-size': 'A4',
    'header-right': '[page]'
}
#вызываем генерацию pdf документа, сохраняем его как ege.best.pdf
#в качестве параметров передаём config и options
pdfkit.from_string(html, 'E:/ittensive/assets/ege.best.pdf',
                   configuration=config, options=options)
#приступаем к отправке письма, создаём новый объект MIMEMultipart()
letter = MIMEMultipart()
#задаём ему поля, отправитель, тему, тип, и адресс отправки 
letter["From"] = "Имя"
letter["Subject"] = "Результаты по ЕГЭ в Москве"
letter["Content-Type"] = "text/html; charset=utf-8"
letter["To"] = "support@ittensive.com"
#к телу письма  прикрепим наш html - документ
letter.attach(MIMEText(html, "html"))
#в качестве вложения добовляем наш pdf - отчёт
attachement = MIMEBase("application", "pdf")
#открываем его, и получаем из него данные
attachement.set_payload(open("E:/ittensive/assets/ege.best.pdf", "rb").read())
attachement.add_header("Content-Disposition",
                      'attachement; filename="ege.best.pdf"')
encoders.encode_base64(attachement)
letter.attach(attachement)
# user = "XXX"
# password = "XXX"
#подключаемся к почтовому серверу и отправляем письмо на адрес support@ittensive.com
#также указываем логин и пароль
server = smtplib.SMTP_SSL("smtp.yandex.com", 465)
server.login(user, password)
server.sendmail("n@ittensive.com",
               "support@ittensive.com",
               letter.as_string())
server.quit()